### Setup

In [1]:
# this first setup block will pretty much always stay the same
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_multiple_reports, do_fit_scenarios
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220830_scenario_projections.ipynb")

2022/09/02 11:26:41 INFO|============================================================


### Fit an initial scenario through Feb 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20220815_aug_gov_briefing/20220815_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2022-10-15',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-08-30' #'2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/02 11:26:48 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/02 11:26:54 INFO|-------------------------|{} Constructing vaccination projections
2022/09/02 11:26:54 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/02 11:27:08 INFO|----------------------|{} Prepping Model
2022/09/02 11:27:08 INFO|-----------------------|{} Prepping Model
2022/09/02 11:27:59 INFO|----------------------|{} Model prepped for fitting in 51.01667058300001 seconds.
2022/09/02 11:27:59 INFO|----------------------|{} Will fit 32 times
2022/09/02 11:29:31 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/32 completed in 92.60216508300002 seconds: {0: {'co': 0.8773342667871553}, 14: {'co': 0.6853312866729}, 28: {'co': 0.5380569819064223}, 42: {'co': 0.0005455976489553808}, 56: {'co': 0.6816102203195106}, 70: {'co': 0.8911965213118624}}
2022/09/02 11:29:31 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/09/02 

2022/09/02 11:34:10 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/32 completed in 11.894082541999978 seconds: {504: {'co': 0.7968505033119822}, 518: {'co': 0.7914271195070701}, 532: {'co': 0.7702037785988026}, 546: {'co': 0.7247399186584289}, 560: {'co': 0.7571072052733184}, 574: {'co': 0.7523891265853447}}
2022/09/02 11:34:10 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/02 11:34:23 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/32 completed in 11.537725582999997 seconds: {532: {'co': 0.770049231778731}, 546: {'co': 0.7264061257058002}, 560: {'co': 0.7535262433609045}, 574: {'co': 0.7625810955322171}, 588: {'co': 0.7986805620289572}, 602: {'co': 0.7797702478751724}}
2022/09/02 11:34:23 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/02 11:34:37 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/32 completed in 12.630450167000049 seconds: 

2022/09/02 11:38:45 INFO|-----------------------|{'fit_batch': '31', 'run_type': 'fit'}: Running forward sim
2022/09/02 11:38:47 INFO|----------------------|{'fit_batch': '31', 'run_type': 'fit'}: Uploading final results
2022/09/02 11:38:52 INFO|----------------------|{'fit_batch': '31', 'run_type': 'fit'}: spec_id: 3560


### Create and run scenarios from Feb 2022 to present

In [5]:
multiprocess = 6

scenario_params = json.load(open("covid_model/analysis/20220815_aug_gov_briefing/20220815_scenario_params.json"))

model_args = {
    'base_spec_id': model.spec_id#3528 #model.spec_id # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# compute vacc effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for hosp_frac in [0.7, 0.8, 0.9]:
    for school_contact_increase in [1.0, 1.1, 1.2]:
        for beta_mult in [0.95, 1.0, 1.05]:
            hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*hosp_frac), "2022-03-15": (0.34 + 0.66*hosp_frac), "2022-03-30": hosp_frac}
            beta_param_adjustment = [{"param": "betta",
                                      "attrs": {"variant": "ba45"},
                                      "mults": {"2020-01-01": beta_mult},
                                      "desc": "sensitivity analysis for beta"}]
            school_contact = [{"param": "betta",
                               "attrs": {'age': '0-19'},
                               "mults": {"2020-01-01": 1, "2022-08-15": school_contact_increase},
                               "desc": "After August 15th, beta increases by a factor of the school contact increase"}]
            lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5, "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5, "2022-07-30": 0.955 + 0.045*vacc_eff_lt5,
                                          "2022-08-10": 0.95 + 0.05*vacc_eff_lt5},
                                "desc": "Starting June 20th, the share of 0-19 getting vaccinated who are < 5 are weighted by the vaccine effectiveness multiplier for this age group"}]
            scenario_model_args.append({'params_defs': scenario_params + beta_param_adjustment + school_contact + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'beta_mult': beta_mult, 'hosp_frac': hosp_frac, 'school_contact_increase': school_contact_increase}})
            
            

In [6]:
# check how many scenarios there are
len(scenario_model_args)

27

In [7]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/08/16 21:30:56 INFO|============================================================
2022/08/16 21:30:56 INFO|============================================================
2022/08/16 21:30:56 INFO|============================================================
2022/08/16 21:30:56 INFO|============================================================
2022/08/16 21:30:56 INFO|============================================================
2022/08/16 21:30:56 INFO|============================================================
2022/08/16 21:31:01 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.0} Retrieving vaccinations data
2022/08/16 21:31:01 INFO|-----------|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.2} Retrieving vaccinations data
2022/08/16 21:31:01 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1} Retrieving vaccinations data
2022/08/16 21:31:01 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 's

2022/08/16 21:32:23 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '0'}: Transmission control fit 1/4 completed in 17.820320374999994 seconds: {767: {'co': 0.8805088101616122}, 781: {'co': 0.8849161365794993}, 795: {'co': 0.8663456708994486}, 809: {'co': 0.8609522432313147}, 823: {'co': 0.8620605722463729}, 837: {'co': 0.8514466175100209}}
2022/08/16 21:32:23 INFO|---------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '0'}: Running forward sim
2022/08/16 21:32:23 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.2, 'fit_batch': '0'}: Transmission control fit 1/4 completed in 17.781526792000008 seconds: {767: {'co': 0.8805112766114739}, 781: {'co': 0.8849065957717042}, 795: {'co': 0.8663643325930487}, 809: {'co': 0.8610567190789896}, 823: {'co': 0.8626214351035072}, 837: {'co': 0.8533723240588861}}
2022/08/16 21:32:23 INFO|---------|{'beta_mult': 1.05, 'hosp_frac': 0.7,

2022/08/16 21:33:09 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '3'}: Transmission control fit 4/4 completed in 14.11099883300001 seconds: {837: {'co': 0.8571814690040632}, 851: {'co': 0.8421934489263547}, 865: {'co': 0.8662887542474966}, 879: {'co': 0.8540737633297255}, 893: {'co': 0.8523468414068835}, 907: {'co': 0.856122220208128}}
2022/08/16 21:33:09 INFO|---------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '3'}: Running forward sim
2022/08/16 21:33:11 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '3', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342852006267}, 14: {'co': 0.6852948714990484}, 28: {'co': 0.5314458547840805}, 42: {'co': 9.144493615558663e-14}, 56: {'co': 0.7004809655627022}, 70: {'co': 0.8356318976452856}, 84: {'co': 0.8298063624151515}, 98: {'co': 0.8311909470178191}, 112: {'co': 0.8778137703118093}, 126: {'co': 0.8241103

2022/08/16 21:33:23 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '3', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342852006267}, 14: {'co': 0.6852948714990484}, 28: {'co': 0.5314458547840805}, 42: {'co': 9.144493615558663e-14}, 56: {'co': 0.7004809655627022}, 70: {'co': 0.8356318976452856}, 84: {'co': 0.8298063624151515}, 98: {'co': 0.8311909470178191}, 112: {'co': 0.8778137703118093}, 126: {'co': 0.8241103559050622}, 140: {'co': 0.7606632916791646}, 154: {'co': 0.671188012002912}, 168: {'co': 0.8430307687954784}, 182: {'co': 0.7826561785501772}, 196: {'co': 0.8191830539526224}, 210: {'co': 0.772603424743875}, 224: {'co': 0.7713676625090525}, 238: {'co': 0.7185596154749083}, 252: {'co': 0.7222209090154319}, 266: {'co': 0.6848822711002804}, 280: {'co': 0.7199443291943943}, 294: {'co': 0.7575590236006461}, 308: {'co': 0.8226616037796585}, 322: {'co': 0.8113710040107285}, 336: {'co': 0.7825986990933562}, 350: {'co': 0.7952256607313511

2022/08/16 21:33:37 INFO|============================================================
2022/08/16 21:33:38 INFO|-----------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 'school_contact_increase': 1.0} Retrieving vaccinations data
2022/08/16 21:33:39 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.8, 'school_contact_increase': 1.0} Retrieving vaccinations data
2022/08/16 21:33:40 INFO|-----------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 'school_contact_increase': 1.0} Constructing vaccination projections
2022/08/16 21:33:40 INFO|-----------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 'school_contact_increase': 1.0} Retrieving hospitalizations data
2022/08/16 21:33:41 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.8, 'school_contact_increase': 1.0} Constructing vaccination projections
2022/08/16 21:33:41 INFO|-----------|{'beta_mult': 0.95, 'hosp_frac': 0.8, 'school_contact_increase': 1.0} Retrieving hospitalizations data
2022/08/16 21:33:41 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.8, 's

...

2022/08/16 21:40:12 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.0, 'fit_batch': '0'}: Transmission control fit 1/4 completed in 18.23684041599995 seconds: {767: {'co': 0.8891171252915077}, 781: {'co': 0.8864378675864217}, 795: {'co': 0.8690639476903383}, 809: {'co': 0.8625050266878825}, 823: {'co': 0.8657146149773576}, 837: {'co': 0.8569748140885746}}
2022/08/16 21:40:12 INFO|---------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.0, 'fit_batch': '0'}: Running forward sim
2022/08/16 21:40:14 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Model prepped for fitting in 64.87800683399996 seconds.
2022/08/16 21:40:14 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1} Solving Model ODE
2022/08/16 21:40:16 INFO|--------|{'beta_mult': 0.95, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '0'}: Transmission control fit 1/4 completed in 19.770615291000013 s

2022/08/16 21:40:55 INFO|--------|{'beta_mult': 1.0, 'hosp_frac': 0.8, 'school_contact_increase': 1.1, 'fit_batch': '3', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342852006267}, 14: {'co': 0.6852948714990484}, 28: {'co': 0.5314458547840805}, 42: {'co': 9.144493615558663e-14}, 56: {'co': 0.7004809655627022}, 70: {'co': 0.8356318976452856}, 84: {'co': 0.8298063624151515}, 98: {'co': 0.8311909470178191}, 112: {'co': 0.8778137703118093}, 126: {'co': 0.8241103559050622}, 140: {'co': 0.7606632916791646}, 154: {'co': 0.671188012002912}, 168: {'co': 0.8430307687954784}, 182: {'co': 0.7826561785501772}, 196: {'co': 0.8191830539526224}, 210: {'co': 0.772603424743875}, 224: {'co': 0.7713676625090525}, 238: {'co': 0.7185596154749083}, 252: {'co': 0.7222209090154319}, 266: {'co': 0.6848822711002804}, 280: {'co': 0.7199443291943943}, 294: {'co': 0.7575590236006461}, 308: {'co': 0.8226616037796585}, 322: {'co': 0.8113710040107285}, 336: {'co': 0.7825986990933562}, 350: {'co': 0.7952256607313511

2022/08/16 21:41:24 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '3'}: Transmission control fit 4/4 completed in 16.21943133299999 seconds: {837: {'co': 0.8641008814072972}, 851: {'co': 0.8559201354402584}, 865: {'co': 0.8859430868295008}, 879: {'co': 0.8837401014703833}, 893: {'co': 0.8865492555215055}, 907: {'co': 0.8915595669267085}}
2022/08/16 21:41:24 INFO|---------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '3'}: Running forward sim
2022/08/16 21:41:25 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.1, 'fit_batch': '3', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8773342852006267}, 14: {'co': 0.6852948714990484}, 28: {'co': 0.5314458547840805}, 42: {'co': 9.144493615558663e-14}, 56: {'co': 0.7004809655627022}, 70: {'co': 0.8356318976452856}, 84: {'co': 0.8298063624151515}, 98: {'co': 0.8311909470178191}, 112: {'co': 0.8778137703118093}, 126: {'co': 0.824110

2022/08/16 21:43:23 INFO|============================================================
2022/08/16 21:43:25 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Retrieving vaccinations data
2022/08/16 21:43:27 INFO|--------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'fit_batch': '3'}: Transmission control fit 4/4 completed in 14.590070458000014 seconds: {837: {'co': 0.8641008983796327}, 851: {'co': 0.8559201047119578}, 865: {'co': 0.8859431040189815}, 879: {'co': 0.8837401076531737}, 893: {'co': 0.8865492122394892}, 907: {'co': 0.8915596498516407}}
2022/08/16 21:43:27 INFO|---------|{'beta_mult': 1.05, 'hosp_frac': 0.9, 'school_contact_increase': 1.2, 'fit_batch': '3'}: Running forward sim
2022/08/16 21:43:28 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase': 1.2} Constructing vaccination projections
2022/08/16 21:43:28 INFO|-----------|{'beta_mult': 1.0, 'hosp_frac': 0.9, 'school_contact_increase':

2022/08/16 21:48:02 INFO|-----------------------|spec_ids: 3529,3530,3531,3532,3533,3534,3535,3536,3537,3538,3539,3540,3541,3542,3543,3544,3545,3546,3547,3548,3549,3550,3551,3552,3553,3554,3555


### Run the report for each fit model

In [8]:
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45'], from_date='2022-01-01')

2022/08/16 21:51:07 INFO|============================================================
2022/08/16 21:51:07 INFO|--------|Solving model
2022/08/16 21:51:08 INFO|============================================================
2022/08/16 21:51:08 INFO|--------|Solving model
2022/08/16 21:51:10 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/08/16 21:51:10 INFO|============================================================
2022/08/16 21:51:10 INFO|--------|Solving model
2022/08/16 21:51:11 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/08/16 21:51:12 INFO|============================================================
2022/08/16 21:51:12 INFO|--------|Solving model
2022/08/16 21:51:13 WARNING|No artists with labels found to put in legend.  Note that artists 

2022/08/16 21:55:50 INFO|============================================================
2022/08/16 21:55:50 INFO|--------|Solving model
2022/08/16 21:55:52 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [9]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-10-15', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2022-10-15', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/08/16 22:57:53 INFO|Projecting
2022/08/16 22:57:53 INFO|
2022/08/16 22:58:15 INFO|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '3', 'run_type': 'fit'}: Running forward sim
2022/08/16 22:58:17 INFO|
2022/08/16 22:58:26 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '3', 'run_type': 'fit'}: Running forward sim
2022/08/16 22:58:27 INFO|
2022/08/16 22:58:36 INFO|{'beta_mult': 1.05, 'hosp_frac': 0.7, 'school_contact_increase': 1.0, 'fit_batch': '3', 'run_type': 'fit'}: Running forward sim
2022/08/16 22:58:37 INFO|
2022/08/16 22:58:46 INFO|{'beta_mult': 0.95, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '3', 'run_type': 'fit'}: Running forward sim
2022/08/16 22:58:47 INFO|
2022/08/16 22:59:07 INFO|{'beta_mult': 1.0, 'hosp_frac': 0.7, 'school_contact_increase': 1.1, 'fit_batch': '3', 'run_type': 'fit'}: Running forward sim
2022/08/16 22:59:09 INFO|
2022/08/16 22:59:18 INFO|{'beta_mult': 1.05, 